# Crypto-Tracker

 We will attempt to collect data of cryptocurrencies and monitor their growth. This will be accomplished by making API calls to https://min-api.cryptocompare.com. We will be tracking the growth of Bitcoin, Ethereum, Ethereum Classic, Ripple, Litecoin, Dash, NEO, EOS and Lisk. The methods are created such that everytime the notebook runs, a date validation technique is used to verify that we do not write the same values to disk (one write per day).   

In [6]:
# import dependancies
import requests     # making calls to the api
import datetime     # date stamp for csv file

In [7]:
# declare global variables
currencies = []
json_response = ''

In [8]:
def complete_api_calls():
    ''' 
    We will be conducting calls for Bitcoin, Ethereum, Ripple, Litecoin, Dash, 
    Ethereum Classic, NEO, EOS and Lisk
    '''
    global currencies
    currencies = [['Bitcoin', 'BTC'], ['Ethereum', 'ETH'], ['RIPPLE', 'XRP'], 
                  ['Litecoin', 'LTC'], ['Dash', 'DASH'], ['Ethereum Classic', 'ETC'], 
                  ['NEO', 'NEO'], ['EOS', 'EOS'], ['Lisk', 'LSK']]

    symbols_for_api_call = ''
    for currency in currencies:
        symbols_for_api_call += currency[1] + ','
    symbols_for_api_call = symbols_for_api_call[0:len(symbols_for_api_call)-1] # last comma
    
    # prepare url before making api call
    url = 'https://min-api.cryptocompare.com/data/pricemulti?fsyms='+symbols_for_api_call
    url += '&tsyms=CAD,USD,BTC,USD'
    response = requests.get(url)
    
    if response.status_code == 200:     # everthing went okay
        global json_response
        json_response = response.json()      # decode to JSON form
        print(json_response)


In [14]:
def create_csv_of_collected_data():
    '''
    Make additional api calls for last close and write all values to file
    '''
    now = datetime.datetime.now()
    date = str(now.day) + '-' + str(now.month) + '-' + str(now.year) + ", "
    timestamp = int(now.timestamp())
    timestamp -= 86400   # minus one day
    
    f = open('tracker.csv', 'a')
    for currency in currencies:
        name = currency[0] + ', '
        symbol = currency[1] + ', '
        cad_price = str(json_response[currency[1]]['CAD'])
        usd_price = str(json_response[currency[1]]['USD']) + ', '
        btc_price = str(json_response[currency[1]]['BTC']) + ', '
        delta_dollars = 0
        delta_percentage = 0
        
        previous_close_url = 'https://min-api.cryptocompare.com/data/pricehistorical?fsym='
        previous_close_url += currency[1] + '&tsyms=CAD&ts=' + str(timestamp)
        previous_close_response = requests.get(previous_close_url)
        if previous_close_response.status_code == 200:
            previous_close_json = previous_close_response.json()
            prev_close = previous_close_json[currency[1]]['CAD']
            delta_dollars = float(cad_price) - float(prev_close)
            delta_percentage = float(cad_price) / float(prev_close) - 1
        
        line = date+name+symbol+cad_price+', '+usd_price+btc_price
        line += '{:.3f} {:.5f}%'.format(delta_dollars, delta_percentage)
        print(line)
        
        for letter in line:
            f.write(letter)
        f.write('\n')
        
    f.close()


In [19]:
def main():
    with open('tracker.csv', 'r') as f:
        lines = f.read().splitlines()
        last_line = lines[-1]
        split = last_line.split(',')
        last_date = split[0]
        now = datetime.datetime.now()
        current_date = str(now.day) + '-' + str(now.month) + '-' + str(now.year)
        
        if last_date != current_date:
            complete_api_calls()
            create_csv_of_collected_data()
            
            
if __name__ == '__main__':
    main()    